# 异步调度

JAX使用异步调度来隐藏Python开销。考虑以下代码：

In [1]:
import numpy as np
import jax.numpy as jnp
from jax import random

x = random.uniform(random.PRNGKey(0), (1000, 1000))
jnp.dot(x, x) + 3

DeviceArray([[258.01974, 249.6486 , 257.13367, ..., 236.67946, 250.68948,
              241.36853],
             [265.65985, 256.28912, 262.1825 , ..., 242.03188, 256.1676 ,
              252.44131],
             [262.38904, 255.72743, 261.2306 , ..., 240.8356 , 255.41084,
              249.62466],
             ...,
             [259.15814, 253.09195, 257.72174, ..., 242.23877, 250.72672,
              247.16637],
             [271.2267 , 261.91208, 265.33398, ..., 248.26645, 262.0539 ,
              261.33704],
             [257.16138, 254.75424, 259.083  , ..., 241.5985 , 248.626  ,
              243.22357]], dtype=float32)

类似 `jnp.dot(x, x)` 的操作被执行的时候，JAX在将控制权返回给Python程序之前不会等待该操作完成。JAX返回一个 `DeviceArray` 值，这是一个未来值，也就是未来将在加速设备上生成的值，不一定立即可用。我们可以检查 `DeviceArray` 的形状或者类型，而不必等待计算完毕，甚至可以像例子中的加法操作，将其传递给另一个JAX计算。仅当我们实际要求从主机检查数组值的时候（例如通过打印或将其转换为普通的 `numpy.ndarray`），JAX才会强制Python代码等待计算完成。

异步调度非常有用，因为它允许Python代码在加速设备之前运行，从而使Python代码可以脱离关键路径。如果Python代码使设备上的工作入队列的速度快于其执行速度，并且前提是Python代码实际上不需要检查主机上的计算输出，那么Python程序就可以使任意数量的工作入队列，并避免了加速设备等待。

异步调度对微基准测试产生了令人惊讶的后果。

In [2]:
%time jnp.dot(x, x)

CPU times: user 195 µs, sys: 231 µs, total: 426 µs
Wall time: 259 µs


DeviceArray([[255.01974, 246.6486 , 254.13365, ..., 233.67946, 247.68948,
              238.36853],
             [262.65985, 253.28912, 259.1825 , ..., 239.03188, 253.1676 ,
              249.44131],
             [259.38904, 252.72743, 258.2306 , ..., 237.8356 , 252.41084,
              246.62466],
             ...,
             [256.15814, 250.09195, 254.72174, ..., 239.23877, 247.72672,
              244.16637],
             [268.2267 , 258.91208, 262.33398, ..., 245.26645, 259.0539 ,
              258.33704],
             [254.16138, 251.75424, 256.083  , ..., 238.5985 , 245.626  ,
              240.22357]], dtype=float32)

在CPU上进行`1000 x 1000`矩阵乘法仅需275微秒！但是事实证明，异步调度会误导我们，我们不是在对矩阵乘法的执行时间计时，而是对调度的工作计时。为了衡量操作的真实成本，我们必须读取主机上的值（例如将其转换为普通的NumPy数组），或者对 `DeviceArray` 值使用 `block_until_ready()` 方法来等待计算完成。

In [3]:
%time np.asarray(jnp.dot(x, x))

CPU times: user 2.63 ms, sys: 0 ns, total: 2.63 ms
Wall time: 2.37 ms


array([[255.01974, 246.6486 , 254.13365, ..., 233.67946, 247.68948,
        238.36853],
       [262.65985, 253.28912, 259.1825 , ..., 239.03188, 253.1676 ,
        249.44131],
       [259.38904, 252.72743, 258.2306 , ..., 237.8356 , 252.41084,
        246.62466],
       ...,
       [256.15814, 250.09195, 254.72174, ..., 239.23877, 247.72672,
        244.16637],
       [268.2267 , 258.91208, 262.33398, ..., 245.26645, 259.0539 ,
        258.33704],
       [254.16138, 251.75424, 256.083  , ..., 238.5985 , 245.626  ,
        240.22357]], dtype=float32)

In [4]:
%time jnp.dot(x, x).block_until_ready()  

CPU times: user 1.14 ms, sys: 0 ns, total: 1.14 ms
Wall time: 824 µs


DeviceArray([[255.01974, 246.6486 , 254.13365, ..., 233.67946, 247.68948,
              238.36853],
             [262.65985, 253.28912, 259.1825 , ..., 239.03188, 253.1676 ,
              249.44131],
             [259.38904, 252.72743, 258.2306 , ..., 237.8356 , 252.41084,
              246.62466],
             ...,
             [256.15814, 250.09195, 254.72174, ..., 239.23877, 247.72672,
              244.16637],
             [268.2267 , 258.91208, 262.33398, ..., 245.26645, 259.0539 ,
              258.33704],
             [254.16138, 251.75424, 256.083  , ..., 238.5985 , 245.626  ,
              240.22357]], dtype=float32)